In [5]:
import glob

In [7]:
V_dirs = glob.glob("../minimise_results/*")

#fig, axs = plt.subplots(nrows=4*len(V_list), ncols=len(mu_files_test), figsize=(len(mu_files_test)*5, 4*5*len(V_list)))

for V_dir in V_dirs:
    V = float(V_dir.split("/")[-1].split("_")[-1])
    mu_dirs = glob.glob(f'{V_dir}/*')
    
    for mu_dir in mu_dirs:
        mu = float(mu_dir.split("/")[-1].split("_")[-1])
        
        U_files = glob.glob(f'{mu_dir}/U_*.hdf5')
        
        for U_file in U_files:
            print(U_file)

../minimise_results/2.00/-1.00/U_2.69.hdf5
../minimise_results/2.00/-1.00/U_1.43.hdf5
../minimise_results/2.00/-1.00/U_0.80.hdf5
../minimise_results/2.00/-1.00/U_2.21.hdf5
../minimise_results/2.00/-1.00/U_1.90.hdf5
../minimise_results/2.00/-1.00/U_2.84.hdf5
../minimise_results/2.00/-1.00/U_0.17.hdf5
../minimise_results/2.00/-1.00/U_0.48.hdf5
../minimise_results/2.00/-1.00/U_0.95.hdf5
../minimise_results/2.00/-1.00/U_1.58.hdf5
../minimise_results/2.00/-1.00/U_2.06.hdf5
../minimise_results/2.00/-1.00/U_2.53.hdf5
../minimise_results/2.00/-1.00/U_0.01.hdf5
../minimise_results/2.00/-1.00/U_1.27.hdf5
../minimise_results/2.00/-1.00/U_0.64.hdf5
../minimise_results/2.00/-1.00/U_1.11.hdf5
../minimise_results/2.00/-1.00/U_1.74.hdf5
../minimise_results/2.00/-1.00/U_2.37.hdf5
../minimise_results/2.00/-1.00/U_3.00.hdf5
../minimise_results/2.00/-1.00/U_0.32.hdf5
